# Übungsblatt 2 - ITSec II

Tutor: Stefan Machmeier(mailto:stefan.machmeier@uni-heidelberg.de)

In [15]:
import hashlib
import os
import time
import base64
from time import sleep
from binascii import hexlify, unhexlify
from urllib.parse import urlparse, parse_qs
from itertools import zip_longest
import hmac

# Aufgabe 1 (30 Pkt.)

## (a) MD5 Hash Kollision (10 Pkt.)

Erzeugen Sie mit `md5_prime` eine Hash-Kollision mit reduzierte Eingangsgröße von $16$ Byte und Ausgangsgröße von $4$ Byte. Reduzierter Ausgangsgröße bedeutet, dass wir nur die ersten $4$ Bytes eines vollständigen MD5 Hash betrachten, um schnell Kollisionen zu erhalten.

Verwenden Sie dazu die Lib `hashlib.md5()`.


In [2]:
def md5_prime(text, output_bytes):
    ###########################################################################
    # YOUR CODE GOES HERE
    ###########################################################################
    pass

In [1]:
###########################################################################
# YOUR CODE GOES HERE
###########################################################################


## (b) MD5 Hash-Kollision mit gegeben Prefix (15 Pkt.)

Wir haben ausführlich betrachtet, wie eine MD5 Kollision mit reduzierter Ein- und Ausgangsgröße erzeugt werden kann. Nun betrachten wir den Anwendungsfall, dass wir einen gegebenen Prefix einer Nachricht haben und darauf basierend zwei Nachrichten mit gleichem Hash erhalten möchten.

Sie können dazu den Floyd Algorithmus (https://en.wikipedia.org/wiki/Cycle_detection) verwenden.

In [7]:
my_prefix = b'\x85\x33\x47'

In [6]:
def md5_prime(message, x, prefix):
  ###########################################################################
  # YOUR CODE GOES HERE
  ###########################################################################
  return 0

In [8]:
def floyd(x, initial):
  x0 = initial
  m0 = None
  m1 = None

  # Start
  tortoise = md5_prime(x0, x, my_prefix)
  hare = md5_prime(tortoise, x, my_prefix)

  ###########################################################################
  # YOUR CODE GOES HERE
  ###########################################################################
  
  
  #print(f'Nachricht {m0.hex()} und {m1.hex()} haben den gleichen Hash {md5_prime(m0, x, my_prefix)}')

In [9]:
# Floyd mit initialen Wert
floyd(x=4, initial=b"1234")

## Aufgabe 2 SHA-1 und HMAC-SHA1 Implementierung (15 Pkt.)

Bevor wir mit einer praktischen Umsetzung einer Timing Attacke beginnen, schauen wir uns die Umsetzung von HMAC-SHA1 aus der Vorlesung an. Berechnen Sie die HMAC-SHA1 mit `hashlib.sha1()` und der gegebenen Funktion `bxor(a, b, longest=True)`. Ihre Berechnung können Sie jederzeit gegen die `hmac` Python Library validieren lassen. Als Hilfestellung schauen Sie sich den RFC 2104 (https://datatracker.ietf.org/doc/html/rfc2104) an.

**Achtung**: Falls Sie diese Berechnung **nicht** erfolgreich implementieren können, kann in den folgenden Aufgaben die `hmac` Python Library verwendet werden.

In [10]:
# XOR Operation für HMAC-SHA1 Implementierung
def bxor(a, b, longest=True):
    if longest:
        return bytes([ x^y for (x, y) in zip_longest(a, b, fillvalue=0)])
    else:
        return bytes([ x^y for (x, y) in zip(a, b)])

In [13]:
def sha1(data):
    ###########################################################################
    # YOUR CODE GOES HERE
    ###########################################################################
    # Verwenden Sie hashlib.sha1()
    return 0

def hmac_sha1(data, key):
    IPAD = b'\x36'*64
    OPAD = b'\x5C'*64
    ###########################################################################
    # YOUR CODE GOES HERE
    ###########################################################################
    # Verwenden Sie sha1(data) und bxor(a, b, longest=True)
    return 0

In [16]:
# Validieren der Funktion
key = bytes('Test Key', 'UTF-8')
message = bytes("Test Nachricht", 'UTF-8')
digester = hmac.new(key, message, hashlib.sha1)
signature1 = digester.digest()

assert (
    hmac_sha1(b'Test Nachricht', b'Test Key')
    == signature1
)

print(f"Die Signatur ist {signature1}")

AssertionError: 

# Aufgabe 3 Timing Attacke auf HMAC-SHA1 (30 Pkt.)

Hier untersuchen wir eine Timing Attacke auf HMAC-SHA1. Normalerweise wird hierzu ein Webserver benötigt, um einen solchen Angriff auszuführen. Zur Vereinfachung gibt es die Klasse `MockWebserver`, welche einen Webserver imitiert und das benötigte Delay von 50ms einbaut.

### Was sind Timing Attacken?

Eine Timing Attacke in der Kryptografie beschreibt einen "Side-Channel Attack", bei dem der Angreifer versucht, das kryptografische System zu kompromittieren, indem er die Zeitspanne für die Ausführung kryptografischer Algorithmen analysiert.

Ein klassisches Beispiel dazu ist ein String Vergleich auf Byte-Ebene.

```py
if user:
    if key == user.key  # Timing attack vector
      return user
    else
      raise 'Invalid key'
  else
    raise 'No user was found'
```

Für die Validierung des Keys wird jedes Byte einzeln überprüft. Sobald ein Byte abweicht, gibt es die Exception zurück. Falls ein Angreifer die Zeitspanne zwischen Senden und Erhalten errechnet, kann er den Key schrittweise erhalten.

```
# 5 ns
'00000000' = '95624587'
# 5 ns
'01000000' = '95624587'
# 5 ns
'09000000' = '95624587'
...
# 15 ns
'95000000' = '95624587'
# 20 ns
'95010000' = '95624587'
...
# 30 ns
'95620000' = '95624587'
```

### Wie funktioniert eine Timing Attacke auf HMAC-SHA1?

Zum Erstellen einer HMAC-SHA1 Signatur wird ein Schlüssel benötigt. Ein Angreifer, der beliebige Daten zum Kompromittieren in das System einspeisen möchte, muss somit eine korrekte Signatur zu den Daten haben, andernfalls wird seine Anfrage abgelehnt/ignoriert.
Wie im oberen Beispiel erläutert, nutzt er dazu die gemessene Zeitspanne zwischen Senden und Erhalten aus.

Die Schritte für einen solchen Angriff können wie folgt sein:
- Der Angreifer sendet eine Nachricht mit HMAC (eigentlich nur eine Folge von Bytes mit der gleichen Länge wie die HMAC), um die initiale Zeit des Systems zu erhalten.
- Der Angreifer sendet erneut dieselbe Nachricht mit Pseudo-HMAC, dass nun jeden (256) möglichen Wert für das erste Byte der HMAC durchläuft.
- Wenn das System sofort einen Fehler zurückgibt, sollte eine Iterationen mit dem gleichen Wert des ersten Bytes, wie es vom System berechnet wurde, länger brauchen.
Diesen Unterschied kann der Angreifer feststellen und das richtige erste Byte für die richtige HMAC der Nachricht erhalten.
- Der Angreifer sendet dieselbe Nachricht und HMAC, diesmal mit dem bekannten korrekten ersten Byte, und iteriert über das zweite Byte, bis er wieder das Byte findet, das das System veranlasst länger zu rechnen, um mit einem Fehler zu antworten. Der Angreifer kennt nun das zweite Byte der korrekten HMAC.
- Das wird für jedes aufeinanderfolgende Byte im HMAC wiederholt, bis eine gültige HMAC für die vom Angreifer gewählte Nachricht ohne Schlüssel abgeleitet worden ist.

## Timing Attacke

Nun schauen wir uns ein Beispiel an, wie eine solche Timing Attacke ablaufen kann.

Unser `MockWebserver` akzeptiert die HTTP Parameter `file` und `signature`, welche in der Funktion `verify_signature` validiert werden. Die Funktion `verify_signature` hat ein Delay von $50$ ms für Datenbankzugriffe o.ä..
In der Validierung der Signature liegt ein klassischer Fehler, wie er bereits oben beschrieben worden ist.

In [71]:
# Validierung der Signatur
class InvalidSignatureError(Exception):
    pass

def verify_signature(signature, data, key):
    # expected_signature = hmac_sha1(data, key) # Sie können hier Ihre eigene Implementierung testn
    expected_signature = unhexlify(hmac.new(data, key, hashlib.sha1).hexdigest())
    for (sig_byte, expected_byte) in zip(signature, expected_signature):
        if sig_byte != expected_byte:
            # dies ist der "vorzeitige Ausgang":
            # technisch gesehen können wir die Signatur zurückweisen
            # *sobald* wir ein Byte gefunden haben, das abweicht.
            # Dies ist jedoch die Ursache für die Sicherheitslücke
            # die in dieser Challenge ausgenutzt wird.
            raise InvalidSignatureError

        # Hier ist das "delay" von 50 ms
        sleep(0.05)

In [70]:
class MockWebserver:
    def __init__(self):
        self.mac_key = os.urandom(16)

    def handle_query(self, url):
        parsed_query = parse_qs(urlparse(url).query)

        # note the "[0]": function `parse_qs` maps keys to *lists* of values
        # because the HTTP protocol allows the same key to appear several time
        # in a query string
        file = parsed_query['file'][0]
        signature = parsed_query['signature'][0]

        sig_bytes = unhexlify(signature)

        verify_signature(sig_bytes, file.encode(), self.mac_key)

Wir sehen, dass eine falsche Signatur zu einer Exception führt.

In [69]:
# Erstelle Fake Website
websever = MockWebserver()
# Test: Hier berechnen wir eine HMAC-SHA1 für den String `foo`
correct_signature = hmac.new(bytes('test', 'UTF-8'), websever.mac_key, hashlib.sha1).hexdigest()
# Handle Query sollte erfolgreich sein
websever.handle_query(f"http://localhost:9000/test?file=test&signature={correct_signature}")
# Nun erstellen wir eine fehlerhafte Signatur, welche zu einem Fehler in unserem Fake Webserver führen sollte
wrong_signature = correct_signature[:2] + 'fff' + correct_signature[5:]
try:
    websever.handle_query(f"http://localhost:9000/test?file=test&signature={wrong_signature}")
    raise Exception('Expected an "InvalidSignatureError"')
except InvalidSignatureError:
    pass

Schauen wir uns die Zeitmessung an (`measure_verification_time`). Wir sehen, dass für das File `test` die korrekte Signatur mit dem Byte `9f` beginnt, welches in der Validierung von `verify_signature` 54 ms benötigt hat. Alle anderen Bytes die wir erzeugt haben, hatte eine Zeitmessung von 0 ms, da in der Verifikation sofort eine Exception entstand als ein Byte von dem erwarteten Byte abwich. Diese Zeitabweichung hilft uns das erste Byte `9f` der korrekten Signatur zu `file` (welche wir nicht kennen) abzuleiten.

In [22]:
def measure_verification_time(signature, websever):
    start_time = time.perf_counter_ns()
    try:
        websever.handle_query(f"http://localhost:9000/test?file=test&signature={signature}")
        raise Exception('signature was not rejected')
    except InvalidSignatureError:
        pass
    end_time = time.perf_counter_ns()

    duration = end_time - start_time
    # we return duration in milliseconds
    # (time.perf_counter_ns() returns nanoseconds)
    return duration//1_000_000

`timings_first_byte` enthält alle 256 Byte Möglichkeiten für das erste Byte.

In [72]:
timings_first_byte = [
    # see https://docs.python.org/3/library/string.html#formatspec
    # for more information on the formating mini-language (the ":02x" thing)
    (
        measure_verification_time(f'{first_byte:02x}' + '0'*(15*2), websever),
        f'{first_byte:02x}',
    )
    for first_byte in range(256)
]

Wir sehen hier die auffällig lange Zeitspanne von dem Byte `9f`

In [73]:
sorted(timings_first_byte, reverse=True)[:5]

[(54, '9f'), (0, 'ff'), (0, 'fe'), (0, 'fd'), (0, 'fc')]

## (b) Byte Recovery für Timing Attacke (20 Pkt.)

Implementieren Sie die Byte Recovery Methode `recover_next_signature_byte`, welche unseren `MockWebserver` und die bisher wiederhergestellten Bytes erhält.
Iterieren Sie dazu über alle möglichen Bytes und bestimmen Sie die Zeit zwischen Absenden und Erhalten, um alle weiteren korrekten Bytes zu bestimmen.


In [17]:
def recover_next_signature_byte(webserver, already_recovered):
    ###########################################################################
    # YOUR CODE GOES HERE
    ###########################################################################
    pass

Zeigen Sie, dass Ihre Implementierung die ersten 4 Byte der korrekten Signatur erzeugen kann.

In [24]:
print('EXPECTED SIGNATURE:', correct_signature)

# Achtung: Das Berechnen der Signature dauert sehr lange, daher reicht es nur die ersten 4 Byte aufzuzeigen.

counter = 0
recovered_signature = str()
for _ in range(16):
    next_byte = recover_next_signature_byte(websever, recovered_signature)
    recovered_signature += next_byte
    counter += 1
    print(next_byte)
    if counter == 4:
        break

NameError: name 'correct_signature' is not defined

## (c) Warum dauert das Berechnen der darauffolgenden Bytes so lange? (5 Pkt.)


## (d) Welche Mitigationsmaßnahmen können angewandt werden, um eine Timing Attack zu verhindern? (5 Pkt.)